In [ ]:
import pandas as pd
import os
from utils import *
from utils import df_from_filings

In [ ]:
from textblob import TextBlob

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

In [ ]:
df = df_from_filings()
df.shape

In [ ]:
%%time
vader_untrained_polarity = df.apply(lambda row: vader.polarity_scores(row['text']), axis=1)

In [ ]:
%%time
vader_untrained_preprocessed_polarity = df.apply(lambda row: vader.polarity_scores(preprocess(row['text'])), axis=1)


In [ ]:
vader_untrained_polarity = pd.DataFrame(vader_untrained_polarity.values.tolist())
vader_untrained_polarity = vader_untrained_polarity.add_prefix('vader_untrained_polarity_');

vader_untrained_preprocessed_polarity = pd.DataFrame(vader_untrained_preprocessed_polarity.values.tolist())
vader_untrained_preprocessed_polarity = vader_untrained_preprocessed_polarity.add_prefix(
    'vader_untrained_preprocessed_polarity_');

In [ ]:
df = pd.concat([df, vader_untrained_polarity, vader_untrained_preprocessed_polarity], axis=1);

In [ ]:
df.describe()

In [ ]:
#TODO lemma, stem, check if remove frequent is good

In [ ]:
%%time
textblob_sentiment_polarity = df.apply(lambda row: TextBlob(row['text']).sentiment.polarity, axis=1)
textblob_sentiment_subjectivity = df.apply(lambda row: TextBlob(row['text']).sentiment.subjectivity, axis=1)

textblob_sentiment = pd.concat([textblob_sentiment_polarity, textblob_sentiment_subjectivity], axis=1)
textblob_sentiment.columns = ['textblob_sentiment_polarity', 'textblob_sentiment_subjectivity']

df = pd.concat([df, textblob_sentiment], axis=1);

In [ ]:
%%time
textblob_preprocessed_sentiment_polarity = df.apply(lambda row: TextBlob(preprocess(row['text'])).sentiment.polarity,
                                                    axis=1)
textblob_preprocessed_sentiment_subjectivity = df.apply(
    lambda row: TextBlob(preprocess(row['text'])).sentiment.subjectivity, axis=1)

textblob_preprocessed_sentiment = pd.concat(
    [textblob_preprocessed_sentiment_polarity, textblob_preprocessed_sentiment_subjectivity], axis=1)
textblob_preprocessed_sentiment.columns = ['textblob_preprocessed_sentiment_polarity',
                                           'textblob_preprocessed_sentiment_subjectivity']

df = pd.concat([df, textblob_preprocessed_sentiment], axis=1);

In [ ]:
df.describe()

In [ ]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

In [ ]:
%%time
def get_flair_sentiment(text):
    s = flair.data.Sentence(text)
    flair_sentiment.predict(s)
    return s.tag, s.score

df_flair_sentiment = df.apply(lambda row: get_flair_sentiment(row['text'])  , axis=1, result_type='expand')
df_flair_sentiment.columns = ['flair_sentiment_tag', 'flair_sentiment_score']
df = pd.concat([df, df_flair_sentiment], axis=1);

# Save data
df_flair_sentiment.to_pickle("flair_backup_2644.pkl")

In [ ]:
%%time
df_flair_sentiment_preprocessed = df.apply(lambda row: get_flair_sentiment(preprocess(row['text'])), axis=1, result_type='expand')
df_flair_sentiment_preprocessed.columns = ['flair_preprocessed_sentiment_tag', 'flair_preprocessed_sentiment_score']
df = pd.concat([df, df_flair_sentiment_preprocessed], axis=1);

# Save data
df_flair_sentiment_preprocessed.to_pickle("flair_preprocessed_backup_2644.pkl")

In [ ]:
df_temp = df[['flair_sentiment_tag', 'flair_sentiment_score', 'flair_preprocessed_sentiment_tag', 'flair_preprocessed_sentiment_score']]

In [ ]:
# Get statistics for flair method WITHOUT preprocessing
print(df_temp['flair_sentiment_tag'].value_counts())

df_temp['flair_sentiment_score'].describe().to_frame()

In [ ]:
# Get statistics for flair method WITH preprocessing
print(df_temp['flair_preprocessed_sentiment_tag'].value_counts())

df_temp['flair_preprocessed_sentiment_score'].describe().to_frame()

In [ ]:
s = flair.data.Sentence("The dog died")
flair_sentiment.predict(s)
s.tag, s.score